# Training ML Models



### Required Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBClassifier, XGBRegressor, plot_importance

from sklearn.neural_network import MLPRegressor

!pip install shap
import shap






# Loading the relevant data

In [ ]:
# Loading the cleaned metadata, that is ready to be encoded and fed into the model

# Contains the following features: 'username', 'like_count', 'tagged_user_count', 'post_type', 'year',
#'month', 'day', 'weekday', 'hour', 'num_hashtags', 'num_emojis',
#'caption_length', 'has_location'

file = "cleaned_metadata.csv"
posts = pd.read_csv(file)

In [ ]:
# Loading the influencer data, that we need to merge with the post metadata

# Contains the following features: 'Username', 'Category', '#Followers', '#Followees', '#Posts'

influencer_data = "influencers.csv"
influencers = pd.read_csv(influencer_data)

# Merge the influencer data (stored in 'influencers' with the post metadata (stored in 'posts')

### (and creating an additional feature, for the average # likes for each influencer)

In [ ]:
# Creating the additional 'average_like' feature, that tracks the average like count for each influencer
average_likes = posts.groupby('username')['like_count'].mean().reset_index()
average_likes.rename(columns={'like_count': 'average_like_count'}, inplace=True)

In [ ]:
# Merging the 'influencers', 'posts', and 'average_likes' dataframes on the feature 'Username'
merged_df = pd.merge(posts, influencers, left_on='username', right_on='Username', how='inner')
merged_df = pd.merge(merged_df, average_likes, on = 'username', how = 'inner')
merged_df = merged_df.drop(columns= ['Username', 'username', 'Unnamed: 0'])

# Removing Outliers

In [ ]:
# We define a function to remove outliers, based on the IQR method
def remove_outliers(df):

    Q1 = df['like_count'].quantile(0.25)
    Q3 = df['like_count'].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = max(Q1 - 1.5 * IQR, 10)
    upper_bound = Q3 + 1.5 * IQR
    print(Q1)
    print(IQR)
    print(lower_bound)
    print(upper_bound)

    # Filter out outliers based on 'like_count'
    return df[(df['like_count'] >= lower_bound) & (df['like_count'] <= upper_bound)]

In [ ]:
# We apply the filtering to the dataframe
filtered = remove_outliers(merged_df)
filtered = filtered[(filtered['#Followers'] < 50000)] # &(filtered['#Followers'] > 10000)]



In [ ]:
min_like_count = filtered['like_count'].min()
print(min_like_count)

# One-Hot Encoding weekdays and categories

First we one-hot encode the weekdays

In [ ]:
# First we copy the 'filtered' dataframe to the 'one_hot_encoded' dataframe
one_hot_encoded = filtered.copy()

# then we create a weekday map
# Monday is 0 and Sunday is 6
weekday_map = {0: 'Mo', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}

# Now we apply the weekday map to the 'weekday' coloumn of the 'one_hot_encoded' dataframe
one_hot_encoded['weekday'] = one_hot_encoded['weekday'].map(weekday_map)

# We one-hot-encode the weekdays finally
one_hot_encoded = pd.get_dummies(one_hot_encoded, columns = ['weekday'])

Then we one-hot encode the categories

In [ ]:
# One-hot encode the categories
one_hot_encoded = pd.get_dummies(one_hot_encoded, columns=["Category"], prefix=["Category_"])

# Binary-Encoding post_type into is_carousel

In [ ]:
# First we copy the one_hot_encoded copy
encoded = one_hot_encoded.copy()
# Renaming the post_type column to is_carousel and convert each value in the column to 1, if it is 'GraphSidecar' and 0 otherwise
encoded['is_carousel'] = encoded['post_type'].apply(lambda x: 1 if x == 'GraphSidecar' else 0)

# Drop the original post_type column
encoded = encoded.drop(columns=['post_type'])

We convert the columns to floats, as our model won't be able to deal with booleans

In [ ]:
encoded = encoded.astype(float)

# Standardizing numerical features

In [ ]:
pd.set_option('display.max_columns', None)

one_hot_encoded.head()

In [ ]:
# First we copy the encoded dataframe to a new dataframe called 'standardized'
standardized = encoded.copy()

features to standardize: tagged_user_count, year (2016-2019), month (1-12), day (1-31), hour (1-24), num_hashtags, num_emojis, caption_length, #Followers, #Followees, #Posts, average_like_count,

In [ ]:
# Subdividing the numerical columns into columns that are normally distributed, bounded/ equally distributed,
# and columns that are rather skewed/ have a lot of outliers
normally_distributed = ['tagged_user_count']
bounded = ['year', 'month', 'day', 'hour']
skewed = ['num_hashtags', 'num_emojis', 'caption_length', '#Followees', '#Posts', 'average_like_count']

# Define the scalers
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
robust_scaler = RobustScaler()

# Apply StandardScaler to the normally_distributed columns, as this is the most suitable scaler for these columns
standardized[normally_distributed] = standard_scaler.fit_transform(standardized[normally_distributed])

# Apply MinMaxScaler to the bounded/ equally distributed columns, as this is the most suitable scaler for these columns
standardized[bounded] = minmax_scaler.fit_transform(standardized[bounded])

# Apply RobustScaler to the skewed/ high outlier ratio columns, as this is the most suitable scaler for these columns
standardized[skewed] = robust_scaler.fit_transform(standardized[skewed])


# Creating final model inputs

Seperating input features (X) and target variable (y)

In [ ]:
# Create the input feature variable X as all the features escept for the like_count
X = standardized.drop(columns=['like_count'])

# Create the new target variable y as like_count
y = standardized['like_count']

Splitting data into training and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the MLP Regressor model

Cross validation with grid search

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50,), (32, 16)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}
mlp_model = MLPRegressor(max_iter=1000, random_state=42, early_stopping = True)
grid_search = GridSearchCV(estimator=mlp_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=3)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the MLP model with best hyperparameters on the full training test
best_mlp_model = MLPRegressor(max_iter=1000, random_state=42, early_stopping = True, **best_params)
best_mlp_model.fit(X_train, y_train)


With given parameters

In [ ]:
mlp = MLPRegressor(max_iter=1000, random_state=42, early_stopping = True, hidden_layer_sizes=(64, 32, 16),
                   activation='relu',
                   solver='adam', verbose=True)
mlp.fit(X_train, y_train)


# Training the Random Forest model

Training the best RF model with cross validation/ grid search

In [ ]:
# # Define a more comprehensive hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Wider range for exploration
    'max_depth': [3, 4, 6, 8, 10],       # Explore deeper trees
    'min_samples_split': [5, 10, 20],     # Higher values for larger dataset
    'min_samples_leaf': [4, 8, 16],       # Higher values for larger dataset
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
    # Add more hyperparameters as needed (consider feature importance selection)
}
# Choose scoring metric (e.g., mean squared error)
scoring = 'neg_mean_squared_error'  # Use negative MSE because GridSearchCV maximizes the scoring function

# Choose cross-validation strategy (e.g., k-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize Random Forest regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=-1)
grid_search.fit(X_val, y_val)

# Get best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

#Train Random Forest model with best hyperparameters
best_rf = RandomForestRegressor(**best_params, random_state=42)
#best_rf = RandomForestRegressor(bootstrap = True, max_depth = 6, max_features = 'sqrt', min_samples_leaf= 1, min_samples_split = 2, n_estimators =  150, random_state=42)
best_rf.fit(X_train, y_train)

# Training Gradient Boost model

Training the best best model via cross validation/ grid serach

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],  # Wider range for exploration
    'learning_rate': [0.01, 0.02, 0.05],      # Lower learning rates
    'max_depth': [3, 4, 6, 8, 10],           # Explore deeper trees
    'min_samples_split': [10, 20, 50],       # Higher values for larger dataset
    'min_samples_leaf': [8, 16, 32],         # Higher values for larger dataset
    'loss': ['huber', 'squared_error', 'poisson']  # Explore other loss functions
}

# Initialize GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=42)

# Initialize GridSearchCV with GradientBoostingRegressor and hyperparameter grid
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform grid search on the training/ validation set
grid_search.fit(X_train, y_train)

# Get best hyperparameters
best_params = grid_search.best_params_

#Train GradientBoostingRegressor with best hyperparameters on the full training set
best_gb_model = GradientBoostingRegressor(**best_params, random_state=42)
#best_gb_model = GradientBoostingRegressor(learning_rate = 0.05, loss =  'huber' , max_depth =  6, min_samples_leaf =  4, min_samples_split =  10, n_estimators = 100, random_state=42)
best_gb_model.fit(X_train, y_train)

# Evaluating the Performance


### Evaluation Function Definition

In [ ]:
# We define the method that eveluates a given model with the test sets
def eveluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse}')

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error (MAE): {mae}')

    # Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    print(f'Mean Absolute Percentage Error (MAPE): {mape}')

    # R-squared (R2)
    r2 = r2_score(y_test, y_pred)
    print(f'R-squared (R2): {r2}')


    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred, color='green')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.xlabel('Actual values')
    plt.ylabel('Predicted values')
    plt.title('Actual vs Predicted')
    plt.grid(True)
    plt.show()

    # Residual plot
    residuals = y_test - y_pred

    plt.figure(figsize=(10, 5))
    plt.scatter(y_mlp_pred, residuals, color='green')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.grid(True)
    plt.show()

    # Residual Analysis (optional)
    residuals = y_test - y_pred
    plt.figure(figsize=(8, 6))
    plt.scatter(y_mlp_pred, residuals, alpha=0.5, color='seagreen')
    plt.axhline(y=0, color='green', linestyle='--')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.tight_layout()
    plt.show()


### Evaluating the MLP model

In [ ]:
eveluate_model(best_mlp_model, X_test, y_test)

### Evaluating the Random Forest model

In [ ]:
eveluate_model(best_rf_model, X_test, y_test)

### Evaluating the Gradient Boost model

In [ ]:
eveluate_model(best_gb_model, X_test, y_test)

# Analyzing Feature Relevance

### Of the MLP model

For the MLP model we compute the SHAP values, which estimate feature importance for specific features

In [ ]:
# Use a smaller subsample of the data
sample_size = 100  # Size of our subsample
X_train_sample = shap.sample(X_train, sample_size)
X_test_sample = shap.sample(X_test, sample_size)

# Compute SHAP values
explainer = shap.KernelExplainer(mlp.predict, X_train_sample)
shap_values = explainer.shap_values(X_test_sample)

# Plot SHAP values
shap.summary_plot(shap_values, X_test_sample, feature_names=X.columns)

Displaying the SHAP value statistics

In [ ]:
# Convert SHAP values to a DataFrame for easier analysis
shap_values_df = pd.DataFrame(shap_values, columns=features)

# Calculate mean absolute SHAP value for each feature
mean_abs_shap_values = shap_values_df.abs().mean()

# Calculate other statistics as needed
shap_summary = shap_values_df.describe().transpose()

# Print the statistics
print("Mean Absolute SHAP Values:\n", mean_abs_shap_values)
print("\nSHAP Summary Statistics:\n", shap_summary)

Plotting the SHAP values in another format

In [ ]:
# Extract the top 10 features based on mean absolute SHAP values
top_features = mean_abs_shap_values.sort_values(ascending=False).head(10).index

# Plot the top 10 mean_abs_shap_values in descending order
plt.figure(figsize=(10, 6))
plt.bar(range(len(top_features)), mean_abs_shap_values[top_features], align='center', color='seagreen')
plt.xticks(range(len(top_features)), top_features, rotation=45, ha='right')
plt.xlabel('Feature')
plt.ylabel('Mean Absolute SHAP Value')
plt.title('Top 10 Features Importance')
plt.show()


### Of the Random Forest Model

In [ ]:
# Feature Importance
feature_importance = best_rf.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]
top_features = sorted_idx[:10]  # Select top 10 features
feature_names = X_train.columns

plt.figure(figsize=(10, 6))
plt.bar(range(len(top_features)), feature_importance[top_features], align='center', color='seagreen')
plt.xticks(range(len(top_features)), feature_names[top_features], rotation=45, ha='right')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Top 10 Feature Importance')
plt.tight_layout()
plt.show()

# Residual Analysis (optional)
residuals = y_test - y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5, color='seagreen')
plt.axhline(y=0, color='green', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.tight_layout()
plt.show()

### For the Gradient Boost model

In [ ]:
# Feature Importance
feature_importance = best_gb_model.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]
top_features = sorted_idx[:10]  # Select top 10 features
feature_names = X_train.columns

plt.figure(figsize=(10, 6))
plt.bar(range(len(top_features)), feature_importance[top_features], align='center', color='seagreen')
plt.xticks(range(len(top_features)), feature_names[top_features], rotation=45, ha='right')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Top 10 Feature Importance')
plt.tight_layout()
plt.show()

# Residual Analysis (optional)
residuals = y_test - y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5, color = 'seagreen')
plt.axhline(y=0, color='seagreen', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.tight_layout()
plt.show()

# Predicting the impact of optimized posts

Now, we will test how the predictions of the number of likes differ for each weekday

In [ ]:
weekdays = pd.DataFrame({
    'Fri': [1, 0, 0, 0, 0, 0, 0],
    'Mo': [0, 1, 0, 0, 0, 0, 0],
    'Sat': [0, 0, 1, 0, 0, 0, 0],
    'Sun':[0, 0, 0, 1, 0, 0, 0],
    'Thu': [0, 0, 0, 0, 1, 0, 0],
    'Tue':[0, 0, 0, 0, 0, 1, 0],
    'Wed': [0, 0, 0, 0, 0, 0, 1],

})
weekdays.head()

In [ ]:
# First we turn
column_names = X.columns
df_X_test = pd.DataFrame(X_test, columns=column_names)


In [ ]:
# We create new data, where we basically test for each post all possible posting weekdays
df_X_test_wo_wd = df_X_test.drop(columns = ['Mo', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
cartesian_product = df_X_test_wo_wd.merge(weekdays, how='cross')

In [ ]:
# We create a new column called 'predicted_likes' that basically stores the predicted likes
# For every post/ weekday combination that we created in the step before
predictions = mlp.predict(cartesian_product)
df_predictions = pd.DataFrame(predictions.reshape(-1,1))
df_predictions.columns = ['like_count']
predicted_likes = pd.concat([cartesian_product, df_predictions], axis=1)

In [ ]:
# Aggregate the data frame based on the Weekday, such that we always aggregate the mean of like_count
# We do this to see  determine the best day to post
aggregated_df = predicted_likes.groupby(['Mo', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']).agg({'like_count': 'mean'}).reset_index()
aggregated_df

In [ ]:
# For every post, we create now a row, where we change the weekday of the post to 'Wed' as Wednesday seems most promising
best_prediction = df_X_test_wo_wd = df_X_test.drop(columns = ['Mo', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'  ])
# Set the 'Mo', 'Tue', 'Thu', 'Fri', 'Sun' columns to zero
best_prediction[['Mo', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']] = 0
# Set the 'Wed' and and the 'post_type__GraphSidecar' column to 1
best_prediction[['Wed']] = 1
best_prediction_y = mlp.predict(best_prediction)

In [ ]:
# Now, we compute the average of the predicted like_count increases
best_prediction_y_series = pd.Series(best_prediction_y.flatten())
relative_improvement_pd = pd.DataFrame()
relative_improvement_pd['relative_improvement'] = (best_prediction_y_series.values - y_test.values) / y_test.values
mean = relative_improvement_pd.mean()
print(mean)